<a href="https://colab.research.google.com/github/Calista-F/Applied-Machine-Learning/blob/main/20220407_Wavelet_runthrough_Signal_60_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pywt
from skimage.restoration import denoise_wavelet
import matplotlib.pyplot as plt
from pylab import *
from matplotlib.pyplot import figure
plt.rcParams.update({'font.family':'serif'})
import pandas as pd
from google.colab import files

from math import log10, sqrt
import cv2
import numpy as np

In [2]:
from sklearn.metrics import  mean_squared_error, mean_absolute_error, max_error
import math

def get_rmse(y_target, y_computed):
  RMSE = mean_squared_error(y_target, y_computed, squared=False)
  return RMSE

In [3]:
def get_min_rmse(df):
  return df[df['RMSE'] == df['RMSE'].min()]

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
# Load worksheet
repo = gc.open_by_url('https://docs.google.com/spreadsheets/d/1VB7SaGWtdRc8lsV4EXCRuQfwXWYLnf8KbMgmsqezgZs/edit#gid=1342867710')
worksheet = repo.worksheet('CCSN_signal60_dist5.0')

#make a df
CCSN60 = worksheet.get_all_values()
ccsn_df = pd.DataFrame(CCSN60)
ccsn_df.columns = ccsn_df.iloc[0]
ccsn_df = ccsn_df.iloc[1:]

#make numeric
ccsn_df = ccsn_df.apply(pd.to_numeric)

###Table 1. Denoising results for the CCSN signal s20a1o05 at a distance of 5 kpc (SNR = 3.9 dB).

In [ ]:
noisy_norm_df = ccsn_df['signal + noise'].loc[20000:24000]/max(ccsn_df['signal + noise'].loc[20000:24000])
signal_norm_df = ccsn_df['signal template'].loc[20000:24000]/max(ccsn_df['signal + noise'].loc[20000:24000])

plt.plot(noisy_norm_df, label ='Signal embedded in noise')
plt.title('Signal 60, 5pc')
plt.plot(signal_norm_df, label ='Signal template')
plt.legend()

Declare necessary lists for table 1

In [ ]:
#pywt.wavelist()

In [ ]:
metrics = ['RMSE','MAE','MAX']
wavelets = ['sym5','db13','coif1']
sym_wavelets = ['sym'+str(i) for i in range(2,21)]
db_wavelets = ['db'+str(i) for i in range(1,39)]
coif_wavelets = ['coif'+str(i) for i in range(1,18)]
levels = np.arange(1,7,1)
methods = ['BayesShrink','VisuShrink']
indexcol = ['RMSE','MAE','MAX']

Create a main list that appends each line and each metric result

In [ ]:
main_list = []

for m in methods:
  cols = []
  for i in range(len(sym_wavelets)):
    for l in range(len(levels)):
      #Make a dataframe for the denoised signal of the noisy normalized signal
      denoised_df = pd.DataFrame(denoise_wavelet(noisy_norm_df, method = m, mode ='hard', wavelet = sym_wavelets[i], wavelet_levels = levels[l], rescale_sigma = 'True'))
      
      #Create col1- a list that contains the signal properties (which signal template was used), denoising properties, and denoising performance metrics
      col1 = [str(m).upper() +', '+ sym_wavelets[i] + ' level ' + str(levels[l])] 
      col1.append(get_rmse(signal_norm_df, denoised_df))
      col1.append(mean_absolute_error(signal_norm_df, denoised_df))
      col1.append(max_error(signal_norm_df, denoised_df))

      #append this column to cols- a list of columns for the table 
      cols.append(col1)
  main_list.append(cols)

  main_array = np.array(main_list)

In [ ]:
def denoising_metrics(wavelet_family, max_level):

  main_list = []
  for m in methods:
    cols = []
    for i in range(len(wavelet_family)):
      for l in range(len(np.arange(1,max_level,1))):
        #Make a dataframe for the denoised signal of the noisy normalized signal
        denoised_df = pd.DataFrame(denoise_wavelet(noisy_norm_df, method = m, mode ='hard', wavelet = wavelet_family[i], wavelet_levels = np.arange(1,max_level,1)[l], rescale_sigma = 'True'))
        
        #Create col1- a list that contains the signal properties (which signal template was used), denoising properties, and denoising performance metrics
        col1 = [str(m).upper() +', '+ wavelet_family[i] + ' level ' + str(levels[l])] 
        col1.append(get_rmse(signal_norm_df, denoised_df))
        col1.append(mean_absolute_error(signal_norm_df, denoised_df))
        col1.append(max_error(signal_norm_df, denoised_df))

        #append this column to cols- a list of columns for the table 
        cols.append(col1)
    main_list.append(cols)
  return np.array(main_list)

In [ ]:
df_minrmse = pd.DataFrame()
df_minrmse

In [ ]:
symlet_array = denoising_metrics(sym_wavelets, 7)

In [ ]:
BayesSymdf = pd.DataFrame(data = symlet_array[0,:,1:], columns = metrics, index = symlet_array[0,:,0])
BayesSymdf

In [ ]:
df_minrmse = df_minrmse.append(get_min_rmse(BayesSymdf))
df_minrmse

In [ ]:
VisuSymdf = pd.DataFrame(data = symlet_array[1,:,1:], columns = metrics, index = symlet_array[1,:,0])
VisuSymdf

In [ ]:
df_minrmse = df_minrmse.append(get_min_rmse(VisuSymdf))
df_minrmse

In [ ]:
db_array = denoising_metrics(db_wavelets,6)

In [ ]:
db_array.shape

In [ ]:
BayesDbdf = pd.DataFrame(data = db_array[0,:,1:], columns = metrics, index = db_array[0,:,0])
BayesDbdf

In [ ]:
df_minrmse = df_minrmse.append(get_min_rmse(BayesDbdf))
df_minrmse

In [ ]:
VisuDbdf = pd.DataFrame(data = db_array[1,:,1:], columns = metrics, index = db_array[1,:,0])
VisuDbdf

In [ ]:
df_minrmse = df_minrmse.append(get_min_rmse(VisuDbdf))
df_minrmse

In [ ]:
coif_array = denoising_metrics(coif_wavelets,6)

In [ ]:
coif_array.shape

In [ ]:
Bayescoifdf = pd.DataFrame(data = coif_array[0,:,1:], columns = metrics, index = coif_array[0,:,0])
Bayescoifdf

In [ ]:
df_minrmse = df_minrmse.append(get_min_rmse(Bayescoifdf))
df_minrmse

In [ ]:
Visucoifdf = pd.DataFrame(data = coif_array[1,:,1:], columns = metrics, index = coif_array[1,:,0])
Visucoifdf

In [ ]:
df_minrmse = df_minrmse.append(get_min_rmse(Visucoifdf))
df_minrmse

In [ ]:
best_df = pd.DataFrame(denoise_wavelet(noisy_norm_df, method = 'BayesShrink', mode ='hard', wavelet = 'db12', wavelet_levels = 5, rescale_sigma = 'True'))

In [ ]:
plt.plot(noisy_norm_df.reset_index(drop=True))
plt.plot(signal_norm_df.reset_index(drop=True))
plt.plot(best_df)

In [ ]:
def printsubplots(denoised):
  linewidthplots=0.6
  fig, axs = plt.subplots(1, 2)
  fig.set_size_inches(10, 4)
  axs[0].plot(noisy_norm_df.reset_index(drop=True),'gray', alpha = 0.7, linewidth=linewidthplots, label='Noisy')
  axs[0].plot(signal_norm_df.reset_index(drop=True), 'r', linewidth=linewidthplots, label='Signal')
  axs[0].plot(denoised,'b', linewidth=linewidthplots, label='Denoised')
  axs[0].legend()
  axs[0].set_xlabel('t (s)')
  axs[0].set_ylim(-1.2,1.2)
  
  axs[1].plot((signal_norm_df.reset_index(drop=True)-denoised), 'b', linewidth=linewidthplots)
  axs[1].set_xlabel('t (s)')
  axs[1].set_ylim(-1.2,1.2)
  plt.tight_layout()

def savensave(figname):
  plt.savefig(figname,pad_inches=0,transparent=True, dpi=300)
  files.download(figname)

In [ ]:
bayesdb605 = denoise_wavelet(noisy_norm_df, method = 'BayesShrink', mode ='hard', wavelet = 'db12', wavelet_levels = 5, rescale_sigma = 'True')

In [ ]:
printsubplots(bayesdb605)
savensave("bayesdb605.png")